In [1]:
# import os
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import StellBlanket.SBGeom as SBGeom
from functools import partial
import jax_sbgeom.coils as jsc

from tests.coils.test_coils import _check_single_vectorized
import pyvista as pv
from dataclasses import dataclass

from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh


vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
vmec_file = vmec_files[0]

coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]

Triangle Elements not compiled


In [2]:
class bb:
    param : str
    def __init__(self, param):
        self.param = param

def _get_all_discrete_coils(request):    
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)    
    coilset_jaxsbgeom = [jsb.coils.DiscreteCoil.from_positions(coilset_sbgeom[i].Get_Vertices()) for i in range(coilset_sbgeom.Number_of_Coils())]
    
    return coilset_jaxsbgeom, coilset_sbgeom

def _get_all_fourier_coils_trunc(request):
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)
    coilset_fourier = SBGeom.Coils.Convert_to_Fourier_Coils(coilset_sbgeom, Nftrunc = 20)
    
    coilset_jax = [jsb.coils.FourierCoil(jnp.array(i.Get_Fourier_Cos()), jnp.array(i.Get_Fourier_Sin()), jnp.array(i.Get_Centre())) for i in coilset_fourier]
    return coilset_jax, coilset_fourier
    


dcoils_jax, coils_sbgeom = _get_all_discrete_coils(bb(coil_files[0]))
coils_jax, coils_sbgeom = _get_all_fourier_coils_trunc(bb(coil_files[1]))

In [3]:
centroid_finitesize = jsc.FiniteSizeCoilSet.from_coils(coils_jax, jsc.RotationMinimizedFrame, 100)

(Array([[[-0.44061201,  0.22452427,  0.86916621],
        [-0.44408664,  0.20964274,  0.87111249],
        [-0.46361632,  0.22097667,  0.858038  ],
        ...,
        [-0.59273492,  0.41100298,  0.692634  ],
        [-0.52644525,  0.33187585,  0.78276038],
        [-0.47488298,  0.26084204,  0.84050437]],

       [[-0.78070282,  0.60227979, -0.16661981],
        [-0.80711327,  0.59032988,  0.0088774 ],
        [-0.8201287 ,  0.5215857 ,  0.23523875],
        ...,
        [-0.74866347,  0.59286316, -0.29667539],
        [-0.75710272,  0.58359085, -0.29362765],
        [-0.77020517,  0.58232712, -0.26015213]],

       [[-0.92786316,  0.32381837,  0.18496385],
        [-0.94200676,  0.23390797,  0.24064563],
        [-0.94986527,  0.14467906,  0.27717131],
        ...,
        [-0.86870837,  0.49436858, -0.03074844],
        [-0.89309987,  0.44849673,  0.03497571],
        [-0.91627646,  0.38696841,  0.10340644]],

       ...,

       [[-0.92786316, -0.32381837, -0.18496385],
        [-

In [6]:
ss= jnp.linspace(0.0, 1.0, 100, endpoint=True)

positions = centroid_finitesize.position(ss)

In [7]:
radial_vectors = centroid_finitesize.radial_vector(ss)

frame = centroid_finitesize.finite_size_frame(ss)
finite_size = centroid_finitesize.finite_size(ss, 0.5, 0.5)

finite_size = jnp.moveaxis(finite_size,2,1).reshape(-1, ss.shape[0], 3)

AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
lines = pv.MultiBlock()

for i in range(positions.shape[0]):
    lines.append(pv.Spline(positions[i], positions.shape[1]))  # create a spline through each line's points



lines_fs = pv.MultiBlock()

for i in range(finite_size.shape[0]):
    lines_fs.append(pv.Spline(finite_size[i], finite_size.shape[1]))  # create a spline through each line's points



In [ ]:
plotter = pv.Plotter()


plotter.add_arrows(positions.reshape(-1,3), frame[..., 0,:].reshape(-1,3), mag=1.0, color='red')
plotter.add_arrows(positions.reshape(-1,3), frame[..., 1,:].reshape(-1,3), mag=1.0, color='green')

plotter.add_mesh(lines_fs, color='yellow', line_width=0.2)
plotter.show()

Widget(value='<iframe src="http://localhost:33945/index.html?ui=P_0x709876d03a70_0&reconnect=auto" class="pyvi…